In [86]:
"""
<prompt> ::= (<roomdevice> <operation>) | (<room>+ <deviceoperation>) [<prompt>]  
<roomdevice> ::= <room>+ <device>+ [<roomdevice>]
<deviceoperation> ::= <device>+ <operation> [<deviceoperation>]
"""

class Lexer:
  # 名詞の定義
  devices = ["電気","エアコン","テレビ"]
  rooms = ["リビング","寝室","和室"]
  operations = ["つけ","消し"]
  joints = ["と","の","を","て"]

  def __init__(self):
    pass  
  def is_room(self,token):
    return token in self.rooms
  def is_device(self,token):
    return token in self.devices
  def is_operation(self,token):
    return token in self.operations
  def remove_joints(self,tokens):
    return [v for v in tokens if v not in self.joints]

  # productとactionの組み合わせをparseする
  def deviceoperation(self,tokens):
    if len(tokens) == 0:
      return [],0
    if not self.is_device(tokens[0]):
      return [],0
    
    orders = []
    devices, operations = [], ""

    assert self.is_device(tokens[0])
    devices.append(tokens[0])
    cnt = 1
    while cnt < len(tokens):
      if self.is_device(tokens[cnt]):
        devices.append(tokens[cnt])
        cnt += 1
      else:
        break

    assert self.is_operation(tokens[cnt])
    operation = tokens[cnt]

    for device in devices:
      orders.append([device, operation])
    next_orders, length = self.deviceoperation(tokens[len(devices)+1:])
    return orders + next_orders, len(devices)+1+length

  def roomdevice(self,tokens):
    if len(tokens) == 0:
      return [], 0
    if not self.is_room(tokens[0]):
      return [], 0
    
    orders, rooms, devices = [],[],[]
    cnt = 0
    assert self.is_room(tokens[cnt])
    while cnt < len(tokens):
      if self.is_room(tokens[cnt]):
        rooms.append(tokens[cnt])
        cnt+=1
      else:
        break
    assert self.is_device(tokens[cnt])
    while cnt < len(tokens):
      if self.is_device(tokens[cnt]):
        devices.append(tokens[cnt])
        cnt += 1
      else:
        break
    
    for room in rooms:
      for device in devices:
        orders.append([room, device])
    
    next_orders, length = self.roomdevice(tokens[len(rooms)+len(devices):])
    return orders + next_orders, len(rooms)+len(devices)+length

  def prompt(self,tokens):
    if len(tokens) == 0:
      return [], 0
    assert self.is_room(tokens[0])
    # "<roomdevice> <operation>"パターンか"<room>+ <deviceoperation>"パターンかの判定
    # 冒頭が<room>+ <device>+ <room> という並びであれば"<roomdevice> <operation>"パターン
    # それ以外は"<room>+ <deviceoperation>"パターン
    room_cnt, device_cnt = 0, 0
    while room_cnt<len(tokens):
      if self.is_room(tokens[room_cnt]):
        room_cnt += 1
      else:
        break
    assert self.is_device(tokens[room_cnt])
    while room_cnt+device_cnt < len(tokens):
      if self.is_device(tokens[room_cnt+device_cnt]):
        device_cnt += 1
      else:
        break
    
    orders, length = [], 0
    assert room_cnt+device_cnt < len(tokens), tokens
    assert self.is_room(tokens[room_cnt+device_cnt]) or self.is_operation(tokens[room_cnt+device_cnt]), tokens[room_cnt+device_cnt]
    if self.is_room(tokens[room_cnt+device_cnt]):
      roomdevices, roomdevicelength = self.roomdevice(tokens)
      assert self.is_operation(tokens[roomdevicelength]), "{}, {}".format(tokens, length)
      operation = tokens[roomdevicelength]
      for rd in roomdevices:
        orders.append(rd + [operation])
      length = roomdevicelength+1 #operationの長さ(1)を足す
    elif self.is_operation(tokens[room_cnt+device_cnt]):
      rooms = tokens[:room_cnt]
      deviceoperations,deviceoperationlength = self.deviceoperation(tokens[room_cnt:])
      for room in rooms:
        for do in deviceoperations:
          orders.append([room]+do)
      length = room_cnt+deviceoperationlength
    
    next_orders, next_length = self.prompt(tokens[length:])
    return orders+next_orders, length+next_length
  def exec(self,text):
    tokens = self.remove_joints(text.split())
    orders, _ = self.prompt(tokens)
    return [" ".join(v) for v in orders]



In [87]:
import unittest
prompt_cases = [
  ["リビング 電気 消し" # <room> <device> <operation>
    , ["リビング 電気 消し"] ]
  , ["リビング 電気 エアコン 消し" # <room> <device>+ <operation>
    , ["リビング 電気 消し", "リビング エアコン 消し"]] 
  , ["リビング 和室 電気 消し" #<room>+ <device> <operation>
    , ["リビング 電気 消し", "和室 電気 消し"]]
  , ["リビング 和室 電気 エアコン 消し" #<room>+ <device>+ <operation>
    , ["リビング 電気 消し", "リビング エアコン 消し", "和室 電気 消し", "和室 エアコン 消し"]]
  , ["リビング 電気 エアコン 消し テレビ つけ" # <room> (<device>+ <operation>)+
    , ["リビング 電気 消し", "リビング エアコン 消し", "リビング テレビ つけ"]]
  , ["リビング 電気 寝室 和室 エアコン テレビ 消し" # (<room>+ <device>+)+ <operation>
    , ["リビング 電気 消し", "寝室 エアコン 消し", "寝室 テレビ 消し", "和室 エアコン 消し", "和室 テレビ 消し"] ]
  , ["リビング 電気 消し 和室 電気 つけ" # (<room> <device> <operation>)+
    , ["リビング 電気 消し","和室 電気 つけ"]]
  , ["リビング 寝室 電気 エアコン 消し 和室 電気 つけ エアコン 消し" # ( ( (<room>+ <device>+)+ <operation> ) | (<room>+ (<device>+ <operation>)+ ) )+
    , ["リビング 電気 消し", "リビング エアコン 消し", "寝室 電気 消し", "寝室 エアコン 消し", "和室 電気 つけ", "和室 エアコン 消し"]]
  , ["リビング の 電気 と エアコン を 消し て" # with joints
    , ["リビング 電気 消し", "リビング エアコン 消し"] ]

  ]
lexer = Lexer()
class TestPrompt(unittest.TestCase):
  #def test_deviceoperation(self):
  #  for arg, result in deviceoperation_cases:
  #    self.assertCountEqual(lexer.deviceoperation(lexer.remove_joints(arg.split()))[0], result)
  def test_exec(self):
    for arg, result in prompt_cases:
      self.assertCountEqual(lexer.exec(arg), result) #要素の数え上げで比較することで順番を無視

if __name__ == '__main__':
  unittest.main(argv=['first-arg-is-ignored'], exit=False) # notebookで実行の場合
  #unittest.main() # scriptで実行の場合

F
FAIL: test_exec (__main__.TestPrompt)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/qz/d2c_grsx75x9b2k6ddzdzm400000gn/T/ipykernel_59986/343249915.py", line 30, in test_exec
    self.assertCountEqual(lexer.exec(arg), result) #要素の数え上げで比較することで順番を無視
  File "/var/folders/qz/d2c_grsx75x9b2k6ddzdzm400000gn/T/ipykernel_59986/2222920161.py", line 125, in exec
    orders, _ = self.prompt(tokens)
  File "/var/folders/qz/d2c_grsx75x9b2k6ddzdzm400000gn/T/ipykernel_59986/2222920161.py", line 108, in prompt
    assert self.is_operation(tokens[length]), "{}, {}".format(tokens, length)
AssertionError: ['リビング', '電気', '寝室', '和室', 'エアコン', 'テレビ', '消し'], 0

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (failures=1)
